# Tensorflow test notebook

This notebook aims to provide a very basic testing perspective on Jupyter notebooks with GPU support, in such way that:

1. Verify the installed Python version
2. Find the GPU on the devices list
3. Check if nvidia-smi loads properly
4. CUDA/ROCm drivers are installed
5. TensorFlow quickstart for beginners
6. TensorFlow tests (basic operations on GPU)
7. TensorBoard spins up properly

## 1. Verify the installed Python version
Multiple notebooks are available, and it can happen of system upgrades, notebooks built with different Python versions, across other possible changes.

The following test will only print out the Python version installed on this notebook, so it can be verified that the expected Python is really running.

> Note: this is yet a manual test, you need to know what version is supposed to run here and match with the output below.

In [2]:
import sys
print(sys.version)

3.12.9 (main, Jun 20 2025, 00:00:00) [GCC 11.5.0 20240719 (Red Hat 11.5.0-5)]


## 2. Find the GPU on the devices list
To understand if the GPUs are present in the current setup, we wil rely on TensorFlow Python client, which refers to the official Python API for interacting with the system's properties and devices.

- If the following code returns a list with items inside, this means that there are GPUs running on this server;
- If the following code returns an empty list, this means that there are no GPUs available;

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    """
    Get all devices connected to this server and return only
    the devices that contain the keyword "GPU"
    """
    local_device_protos = device_lib.list_local_devices()
    return ([x.name for x in local_device_protos if x.device_type == 'GPU'])

get_available_gpus()

## 3. Check if nvidia-smi loads properly
The NVIDIA System Management Interface (nvidia-smi) is a command line utility, based on top of the NVIDIA Management Library (NVML), intended to aid in the management and monitoring of NVIDIA GPU devices.

The following command only spins up the `nvidia-smi` utility:

In [ ]:
!nvidia-smi

## 4. CUDA/ROCm drivers are installed;
This test aims to simply check if CUDA/ROCm drivers are properly installed. To test this, the `nvcc` command will be executed for NVIDIA GPUs and `hipcc` for ROCm GPUs.

> Note: the code is as simple as possible, run the ones that makes sense for the tests you are doing (there are no extended programming to check automatically, etc, this is done this way on purpose to simplify the code as much as possible

In [ ]:
# Run the following for NVIDIA GPUs
!nvcc --version

In [ ]:
# Run the following for ROCm GPUs
!hipcc --version

## 5. TensorFlow quickstart for beginners
This test aims to use the basic Getting Started tutorial on TensorFlow website to check if this installation is working properly:

- Load a prebuilt dataset;
- Build a neural network machine learning model that classifies images;
- Train this neural network;
- Evaluate the accuracy of the model;

The expected output here is a matrix of probabilities, i.e.:

```
<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.6603454e-07, 1.1202768e-09, 3.5647324e-06, 2.7514023e-05,
        1.0256378e-10, 1.7054673e-07...
```

In [ ]:
import tensorflow as tf

# 1. Load a dataset
# ----------------------------------------------------------------------------------------------
# Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255.
# Scale these values to a range of 0 to 1 by dividing the values by 255.0.
# This also converts the sample data from integers to floating-point numbers:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


# 2. Build a machine learning model
# ----------------------------------------------------------------------------------------------
# Build a tf.keras.Sequential model:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# Sequential is useful for stacking layers where each layer has one input tensor and one output tensor.
# Layers are functions with a known mathematical structure that can be reused and have trainable variables.
# Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.
#
# For each example, the model returns a vector of logits or log-odds scores, one for each class.
predictions = model(x_train[:1]).numpy()
predictions

# The tf.nn.softmax function converts these logits to probabilities for each class:
tf.nn.softmax(predictions).numpy()

# Define a loss function for training using losses.SparseCategoricalCrossentropy:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example.
# This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.
#
# This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.
loss_fn(y_train[:1], predictions).numpy()

# Before you start training, configure and compile the model using Keras Model.compile.Set the optimizer class to adam, set the loss to the loss_fn
# function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])


# 3. Train and evaluate your model
# ----------------------------------------------------------------------------------------------
# Use the Model.fit method to adjust your model parameters and minimize the loss:
model.fit(x_train, y_train, epochs=5)

# The Model.evaluate method checks the model's performance, usually on a validation set or test set.
model.evaluate(x_test,  y_test, verbose=2)

# The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.
#
# If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
probability_model(x_test[:5])

## 6. TensorFlow tests (basic operations on GPU)
This test aims to try out the GPU basic commands, like `add`, `multiply`, `matmul`, `reduce_sum` and `reduce_mean` just to be sure that TensorFlow is really running well.

For more information of the methods that will be tested here:
- [tf.add](https://www.tensorflow.org/api_docs/python/tf/math/add)
- [tf.multiply](https://www.tensorflow.org/api_docs/python/tf/math/multiply)
- [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/linalg/matmul)
- [tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/math/reduce_sum)
- [tf.reduce_mean](https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean)

In [ ]:
import tensorflow as tf
import numpy as np

# Basic arithmetic
a = tf.constant([1.0, 2.0, 3.0, 4.0])
b = tf.constant([4.0, 3.0, 2.0, 1.0])

# Addition
add_result = tf.add(a, b)
print(f"Addition: {a.numpy()} + {b.numpy()} = {add_result.numpy()}")

# Multiplication
mul_result = tf.multiply(a, b)
print(f"Multiplication: {mul_result.numpy()}")

# Matrix operations
matrix_a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
matrix_b = tf.constant([[2.0, 1.0], [1.0, 3.0]])
matmul_result = tf.matmul(matrix_a, matrix_b)
print(f"Matrix multiplication result: \n{matmul_result.numpy()}")

# Reduction operations
reduce_sum = tf.reduce_sum(a)
reduce_mean = tf.reduce_mean(a)
print(f"Sum: {reduce_sum.numpy()}, Mean: {reduce_mean.numpy()}")

## 7. TensorBoard spins up properly
[TensorBoard](https://www.tensorflow.org/tensorboard) provides the visualization and tooling needed for machine learning experimentation:

- Tracking and visualizing metrics such as loss and accuracy
- Visualizing the model graph (ops and layers)
- Viewing histograms of weights, biases, or other tensors as they change over time
- Projecting embeddings to a lower dimensional space
- Displaying images, text, and audio data
- Profiling TensorFlow programs
- and more...

This basic test aims to check if the TensorBoard UI spins up properly and that there are no major issues (see [RHOAIENG-20553](https://issues.redhat.com/browse/RHOAIENG-20553))

In [ ]:
import os

# Tensorboard needs to run through a proxy as the Jupyter Notebook is running with a proxy / internal connection
# More information: https://medium.com/@adrian.punga_29809/how-to-use-tensorboard-inline-in-jupyter-lab-or-colab-d28519619d28
os.environ["TENSORBOARD_PROXY_URL"] = os.environ["NB_PREFIX"]+"/proxy/6006/"

# Load TensorBoard extension
%load_ext tensorboard

# Show TensorBoard in the Jupyter Notebook
%tensorboard --logdir /opt/app-root/src/shared